<!-- ## import the csv of data from a Wetlands Insight Tool plot
## render the graph
## select the time of interest
## plot the spatial distribution of the WIT classification for the time of interest -->


- import csv of a given WIT output
- render graph
- select point on graph by clicking
- plot spatial wit for point
- output to GeoTiff

In [17]:
 %matplotlib widget

In [32]:
#run bokeh
from bokeh.plotting import figure,show
from bokeh.io import output_notebook #outputs bokeh to notebook
output_notebook() #this is the command to tell bokeh to output into the jupyterlab notebook

Loading BokehJS ...

In [20]:
import datacube
import datetime
# import fiona
import geopandas as gpd
import os
import glob
import numpy as np
import pandas as pd

import rasterio.mask
import rasterio.features
import seaborn as sns
# from shapely import geometry
import sys
import xarray as xr

from datetime import datetime, timedelta
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from textwrap import wrap

from datacube.storage import masking
# from datacube.utils import geometry
# from datacube.utils.geometry import CRS
# from digitalearthau.utils import wofs_fuser

sys.path.append('/g/data/r78/rjd547/jupyter_notebooks/dea-notebooks/10_Scripts')
import DEADataHandling, DEAPlotting, TasseledCapTools, WetlandsTools, SpatialTools, wetlands_insight

dc = datacube.Datacube(app='wetlands insight tool')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
#set input directory
# global Input_dir
Input_dir = '/g/data/r78/rjd547/Ramsar_Wetlands/Wetlands_Insight_Tool/'

### list available sites to plot

In [22]:
#change to the input directory
os.chdir(Input_dir)

In [24]:
# #print all files
# for file in glob.glob("*.csv"):
#     print(file)  

In [25]:
#alternatively, print the files you want only
search_term = 'Hattah'
for file in glob.glob("*Hattah*.csv"):
    print(file)

Hattah-Kulkyne_Lakes-Lake_Kramen-VIC-37.csv
Hattah-Kulkyne_Lakes-Lake_Brockie-VIC-32.csv
Hattah-Kulkyne_Lakes-Lake_Yelwell-VIC-40.csv
Hattah-Kulkyne_Lakes-Lake_Hattah-VIC-35.csv
Hattah-Kulkyne_Lakes-Lake_Mournpall-VIC-39.csv
Hattah-Kulkyne_Lakes-Lake_Cantala-VIC-34.csv
Hattah-Kulkyne_Lakes-Lake_Bitterang-VIC-31.csv
Hattah-Kulkyne_Lakes-Lake_Lockie-VIC-38.csv
Hattah-Kulkyne_Lakes-Lake_Yerang-VIC-41.csv
Hattah-Kulkyne_Lakes-Lake_Bulla-VIC-33.csv
Hattah-Kulkyne_Lakes-Lake_Konardin-VIC-36.csv
Hattah-Kulkyne_Lakes-Lake_Arawak-VIC-30.csv


In [26]:
#read in your desired csv of the WIT data. Rename the columns so they are easier to understand and plot 
WITdata = pd.read_csv('/g/data/r78/rjd547/Ramsar_Wetlands/Wetlands_Insight_Tool/Hattah-Kulkyne_Lakes-Lake_Hattah-VIC-35.csv',
                     index_col=0, infer_datetime_format=True)

#give the index a name that reflects that it is time, measured in UTC not AEDT/AEST
WITdata.index.names = ['utc_time']

#format the index of the dataframe as a date, not as a string
WITdata.index = pd.to_datetime(WITdata.index)
#Rename the columns so they are easier to understand and plot
WITdata = WITdata.rename(columns={"wofs_area_percent" : "water", 
                        "tcw_area_percent" : "wet",
                       "PV_percent" : "green",
                       "NPV_percent" : "dry",
                       "BS_percent" : "bare"}) 
WITdata.head()

WITarray = WITdata.to_xarray()

In [27]:
WITarray.utc_time[0].data

array('1986-08-17T23:42:44.500000000', dtype='datetime64[ns]')

In [28]:
#set up colour palette for our plots
pal = [sns.xkcd_rgb["cobalt blue"],
       sns.xkcd_rgb["neon blue"],
       sns.xkcd_rgb["grass"],
       sns.xkcd_rgb["beige"],
       sns.xkcd_rgb["brown"]]  

Loading BokehJS ...

In [96]:
WITdata.index[0]

Timestamp('1986-08-17 23:42:44.500000')

In [85]:
WITdata(0)

TypeError: 'DataFrame' object is not callable

In [98]:


from bokeh.plotting import figure, output_file, show
from bokeh.palettes import brewer
from bokeh.models.formatters import DatetimeTickFormatter


def prettierWITplot():
    '''Function to use bokeh to draw a WIT plot.'''
    print('stuff')
    

#set up the x axis to recognise date and time. Note that you will only see the days when you zoom in.
p =figure(x_axis_type='datetime')
p.xaxis.formatter=DatetimeTickFormatter(days=["%d/%m/%Y"])
p.xaxis.major_label_orientation = 45

p.varea_stack(['water',
              'wet',
              'green',
              'dry',
              'bare'], x= 'utc_time', color=brewer['Spectral'][5], source = WITdata)

from bokeh.models import Range1d #use this to set the newWIT graph ranges

left, right, bottom, top = WITdata.index[0], WITdata.index[-1], 0, 100
p.x_range=Range1d(left, right)
p.y_range=Range1d(bottom, top)
    
show(p)   

In [15]:
#     WITarray.utc_time.data, 
#               WITarray.water, 
#               WITarray.wet, 
#               WITarray.green, 
#               WITarray.dry,
#               WITarray.bare,
#               labels=['water',
#                       'wet',
#                       'green veg',
#                       'dry veg',
#                       'bare soil',
#                      ], colors=pal, alpha = 0.6)

In [16]:
#set axis limits to the min and max
plt.axis(xmin = WITarray.utc_time.data[0], xmax = WITarray.utc_time.data[-1], ymin = 0, ymax = 100)

#add a legend and a tight plot box
plt.legend(loc='lower left', framealpha=0.6)
#plt.tight_layout()

#create rectangle borders for no-data times (SLC-off only)
LS5_8_gap_start = datetime(2011,11,1)
LS5_8_gap_end = datetime(2013,4,1)

# convert to matplotlib date representation
gap_start = mdates.date2num(LS5_8_gap_start)
gap_end = mdates.date2num(LS5_8_gap_end)
gap = gap_end - gap_start


#set date ticks every year
years = mdates.YearLocator(2)
yearsFmt = mdates.DateFormatter('%Y')
ax = plt.gca()
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(yearsFmt)
ax.yaxis.set_ticks(np.arange(0,110,10))
ax.set_xlabel(f'The Fractional Cover algorithm developed by the Joint Remote'
f' Sensing Research Program and \n the Water Observations from Space algorithm '
f'developed by Geoscience Australia are used in the production of this data',style='italic')


#set up rectangle
slc_rectangle= Rectangle((gap_start,0), gap, 100,alpha = 0.5, facecolor=sns.xkcd_rgb['white'],
             edgecolor=sns.xkcd_rgb['white'], hatch="////",linewidth=2)
ax.add_patch(slc_rectangle)
fig.autofmt_xdate()


polyName=search_term+"?"

polyName=polyName.replace("'","\\'")
title=ax.set_title("\n".join(wrap(f'Percentage of area WOfS, Wetness, Fractional Cover for {polyName}')))
fig.tight_layout()
title.set_y(1.05)

#save the figure
#4plt.savefig(f'{Output_dir}{polyName}.png')#, transparent=True)

#create a click event for plotting the graph, to return the date clicked on.
# def on_pick(event):
#     print (f'time selected={event.xdata}')
#     global return_timesel
#     return_timesel=event.xdata
    
    
# cid = fig.canvas.mpl_connect('pick_event', on_pick)
# plt.show()
# print(f'plot created for {polyName}')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


NameError: name 'fig' is not defined

In [ ]:
# #make a stacked area plot
# plt.clf()
# fig= plt.figure(figsize = (14,7))
# plt.stackplot(WITarray.utc_time.data, 
#               WITarray.water, 
#               WITarray.wet, 
#               WITarray.green, 
#               WITarray.dry,
#               WITarray.bare,
#               labels=['water',
#                       'wet',
#                       'green veg',
#                       'dry veg',
#                       'bare soil',
#                      ], colors=pal, alpha = 0.6)

# #set axis limits to the min and max
# plt.axis(xmin = WITarray.utc_time.data[0], xmax = WITarray.utc_time.data[-1], ymin = 0, ymax = 100)

# #add a legend and a tight plot box
# plt.legend(loc='lower left', framealpha=0.6)
# #plt.tight_layout()

# #create rectangle borders for no-data times (SLC-off only)
# LS5_8_gap_start = datetime(2011,11,1)
# LS5_8_gap_end = datetime(2013,4,1)

# # convert to matplotlib date representation
# gap_start = mdates.date2num(LS5_8_gap_start)
# gap_end = mdates.date2num(LS5_8_gap_end)
# gap = gap_end - gap_start


# #set date ticks every year
# years = mdates.YearLocator(2)
# yearsFmt = mdates.DateFormatter('%Y')
# ax = plt.gca()
# ax.xaxis.set_major_locator(years)
# ax.xaxis.set_major_formatter(yearsFmt)
# ax.yaxis.set_ticks(np.arange(0,110,10))
# ax.set_xlabel(f'The Fractional Cover algorithm developed by the Joint Remote'
# f' Sensing Research Program and \n the Water Observations from Space algorithm '
# f'developed by Geoscience Australia are used in the production of this data',style='italic')


# #set up rectangle
# slc_rectangle= Rectangle((gap_start,0), gap, 100,alpha = 0.5, facecolor=sns.xkcd_rgb['white'],
#              edgecolor=sns.xkcd_rgb['white'], hatch="////",linewidth=2)
# ax.add_patch(slc_rectangle)
# fig.autofmt_xdate()


# polyName=search_term+"?"

# polyName=polyName.replace("'","\\'")
# title=ax.set_title("\n".join(wrap(f'Percentage of area WOfS, Wetness, Fractional Cover for {polyName}')))
# fig.tight_layout()
# title.set_y(1.05)

# #save the figure
# #4plt.savefig(f'{Output_dir}{polyName}.png')#, transparent=True)

# #create a click event for plotting the graph, to return the date clicked on.
# def on_pick(event):
#     print (f'time selected={event.xdata}')
#     global return_timesel
#     return_timesel=event.xdata
    
    
# cid = fig.canvas.mpl_connect('pick_event', on_pick)
# plt.show()
# print(f'plot created for {polyName}')

### query: setup query as output of javascript picker from bokeh plot

In [ ]:
query = {'geopolygon': geom,
         #'time': (gauge_data.index[0], gauge_data.index[-1]) #get first and last times for our query from original xlsx
        # 'time':('2018-01-01', '2019-11-01'), 
         }